# BERT

https://zhuanlan.zhihu.com/p/61671334
https://colab.research.google.com/github/wshuyi/demo-chinese-text-binary-classification-with-bert/blob/master/demo_chinese_text_binary_classification_with_bert.ipynb
https://analyticsindiamag.com/step-by-step-guide-to-implement-multi-class-classification-with-bert-tensorflow/

---

### Environment Setting

python 3.7
pip install tensorflow==1.15
pip install --upgrade tensorflow-hub
pip install bert-tensorflow==1.0.1

# BERT model builder

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pickle
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
import random
random.seed(583)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
def pretty_print(result):
    df = pd.DataFrame([result]).T
    df.columns = ["values"]
    return df

In [3]:
def create_tokenizer_from_hub_module(bert_model_hub):
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(bert_model_hub)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])

  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

def make_features(dataset, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN):
    input_example = dataset.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN],
                                                                   text_b = None,
                                                                   label = x[LABEL_COLUMN]), axis = 1)
    features = bert.run_classifier.convert_examples_to_features(input_example, label_list, MAX_SEQ_LENGTH, tokenizer)
    return features

def create_model(bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      bert_model_hub,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(bert_model_hub, num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)

    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics.
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        # f1_score = tf.contrib.metrics.f1_score(
        #     label_ids,
        #     predicted_labels)
        # auc = tf.metrics.auc(
        #     label_ids,
        #     predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels)
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            # "f1_score": f1_score,
            # "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

def estimator_builder(bert_model_hub, OUTPUT_DIR, SAVE_SUMMARY_STEPS, SAVE_CHECKPOINTS_STEPS, label_list, LEARNING_RATE, num_train_steps, num_warmup_steps, LOG_STEP_COUNT_STEPS, BATCH_SIZE):

    # Specify outpit directory and number of checkpoint steps to save
    run_config = tf.estimator.RunConfig(
        model_dir=OUTPUT_DIR,
        save_summary_steps=SAVE_SUMMARY_STEPS,
        log_step_count_steps=LOG_STEP_COUNT_STEPS,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

    model_fn = model_fn_builder(
      bert_model_hub = bert_model_hub,
      num_labels=len(label_list),
      learning_rate=LEARNING_RATE,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps)

    estimator = tf.estimator.Estimator(
      model_fn=model_fn,
      config=run_config,
      params={"batch_size": BATCH_SIZE})
    return estimator, model_fn, run_config

In [4]:
def run_on_dfs(train, test, DATA_COLUMN, LABEL_COLUMN,
               MAX_SEQ_LENGTH = 128,
              BATCH_SIZE = 32,
              LEARNING_RATE = 2e-5,
              NUM_TRAIN_EPOCHS = 3.0,
              WARMUP_PROPORTION = 0.1,
              SAVE_SUMMARY_STEPS = 100,
                SAVE_CHECKPOINTS_STEPS = 10000,
                LOG_STEP_COUNT_STEPS = 100,
              bert_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"):

    label_list = train[LABEL_COLUMN].unique().tolist()

    tokenizer = create_tokenizer_from_hub_module(bert_model_hub)

    train_features = make_features(train, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)
    test_features = make_features(test, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)

    num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

    estimator, model_fn, run_config = estimator_builder(
                                  bert_model_hub,
                                  OUTPUT_DIR,
                                  SAVE_SUMMARY_STEPS,
                                  SAVE_CHECKPOINTS_STEPS,
                                  label_list,
                                  LEARNING_RATE,
                                  num_train_steps,
                                  num_warmup_steps,
                                  LOG_STEP_COUNT_STEPS,
                                  BATCH_SIZE)

    train_input_fn = bert.run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=False)

    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

    test_input_fn = run_classifier.input_fn_builder(
        features=test_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False)

    result_dict = estimator.evaluate(input_fn=test_input_fn, steps=None)
    return result_dict, estimator, test_input_fn, label_list

# Load the dataset

In [5]:
# load the dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score, f1_score
oba_cleanall = np.load('./TrainData/tweets_oba.npy', allow_pickle=True)
targets_oba = np.load('./TrainData/targets_oba.npy', allow_pickle=True)
rom_cleanall = np.load('./TrainData/tweets_rom.npy', allow_pickle=True)
targets_rom = np.load('./TrainData/targets_rom.npy', allow_pickle=True)
df_oba = pd.DataFrame({'label': targets_oba, 'text':oba_cleanall})
df_rom = pd.DataFrame({'label': targets_rom,'text':rom_cleanall})

In [6]:
df_oba.head()

,label,text
0,0,kirkpatrick wore basebal cap embroid barack ob...
1,1,obama debat cracker ass cracker tonight tune t...
2,0,miss point im afraid understand bigger pictur ...
3,-1,wa rais democrat left parti year ago lifetim n...
4,0,obama camp cant afford lower expect tonight de...


In [7]:
df_rom.head()

,label,text
0,-1,insidiousmitt romney bain help philip morri ge...
1,-1,mean like romney cheat primari
2,-1,mitt romney still doesnt believ black presid
3,-1,romney tax plan deserv nd look becaus ha secre...
4,1,hope romney debat prep w peopl last time


In [8]:
df_all = df_oba.append(df_rom)
df_all = df_all.sample(frac=1)
df_all.head()

,label,text
2560,1,govern doe creat job true romney true
3647,0,romney campaign file feder suit ensur militari...
5618,-1,amno romney enough romneydelus get back realiti
3027,-1,waystogetshot vote mitt romney
1819,0,mitt romney avoid militari servic seek receiv ...


In [9]:
myparam = {
        "DATA_COLUMN": "text",
        "LABEL_COLUMN": "label",
        "MAX_SEQ_LENGTH": 30,
        "LEARNING_RATE": 2e-5,
        "SAVE_SUMMARY_STEPS":25,
        "NUM_TRAIN_EPOCHS":3,
        "BATCH_SIZE": 16,
        "LOG_STEP_COUNT_STEPS": 100,
        "bert_model_hub":"https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
#         "bert_model_hub":"https://tfhub.dev/google/bert_uncased_L-24_H-1024_A-16/1"
#         "bert_model_hub":"https://tfhub.dev/digitalepidemiologylab/covid-twitter-bert/2"
    }

# Fine-tuning
### Training: Obama dataset

In [10]:
OUTPUT_DIR = 'output_oba'
train_oba, test_oba = train_test_split(df_oba, test_size=0.2, random_state=42)
result_oba, estimator_oba, test_input_fn_oba, label_list_oba = run_on_dfs(train_oba, test_oba, **myparam)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 4499


INFO:tensorflow:Writing example 0 of 4499


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Finished evaluation at 2020-12-02-20:23:15


INFO:tensorflow:Saving dict for global step 843: eval_accuracy = 0.6071111, false_negatives = 185.0, false_positives = 166.0, global_step = 843, loss = 1.1263611, precision = 0.767507, recall = 0.74761254, true_negatives = 226.0, true_positives = 548.0


INFO:tensorflow:Saving dict for global step 843: eval_accuracy = 0.6071111, false_negatives = 185.0, false_positives = 166.0, global_step = 843, loss = 1.1263611, precision = 0.767507, recall = 0.74761254, true_negatives = 226.0, true_positives = 548.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 843: output_oba\model.ckpt-843


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 843: output_oba\model.ckpt-843


In [11]:
pretty_print(result_oba)

,values
eval_accuracy,0.607111
false_negatives,185.000000
false_positives,166.000000
global_step,843.000000
loss,1.126361
precision,0.767507
recall,0.747613
true_negatives,226.000000
true_positives,548.000000


### Training: Romney dataset

In [12]:
OUTPUT_DIR = 'output_rom'
train_rom, test_rom = train_test_split(df_rom, test_size=0.2, random_state=42)
result_rom, estimator_rom, test_input_fn_rom, label_list_rom = run_on_dfs(train_rom, test_rom, **myparam)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 4518


INFO:tensorflow:Writing example 0 of 4518


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Finished evaluation at 2020-12-02-20:23:33


INFO:tensorflow:Saving dict for global step 847: eval_accuracy = 0.6088496, false_negatives = 176.0, false_positives = 150.0, global_step = 847, loss = 1.1159534, precision = 0.71153843, recall = 0.6776557, true_negatives = 434.0, true_positives = 370.0


INFO:tensorflow:Saving dict for global step 847: eval_accuracy = 0.6088496, false_negatives = 176.0, false_positives = 150.0, global_step = 847, loss = 1.1159534, precision = 0.71153843, recall = 0.6776557, true_negatives = 434.0, true_positives = 370.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 847: output_rom\model.ckpt-847


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 847: output_rom\model.ckpt-847


In [13]:
pretty_print(result_rom)

,values
eval_accuracy,0.608850
false_negatives,176.000000
false_positives,150.000000
global_step,847.000000
loss,1.115953
precision,0.711538
recall,0.677656
true_negatives,434.000000
true_positives,370.000000


### Training: All dataset

In [14]:
OUTPUT_DIR = 'output_all'
train_all, test_all = train_test_split(df_all, test_size=0.2, random_state=42)
result_all, estimator_all, test_input_fn_all, label_list_all = run_on_dfs(train_all, test_all, **myparam)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 9017


INFO:tensorflow:Writing example 0 of 9017


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Finished evaluation at 2020-12-02-20:28:20


INFO:tensorflow:Saving dict for global step 1690: eval_accuracy = 0.6186253, false_negatives = 311.0, false_positives = 306.0, global_step = 1690, loss = 1.1192281, precision = 0.7631579, recall = 0.7602159, true_negatives = 652.0, true_positives = 986.0


INFO:tensorflow:Saving dict for global step 1690: eval_accuracy = 0.6186253, false_negatives = 311.0, false_positives = 306.0, global_step = 1690, loss = 1.1192281, precision = 0.7631579, recall = 0.7602159, true_negatives = 652.0, true_positives = 986.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1690: output_all\model.ckpt-1690


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1690: output_all\model.ckpt-1690


In [15]:
pretty_print(result_all)

,values
eval_accuracy,0.618625
false_negatives,311.000000
false_positives,306.000000
global_step,1690.000000
loss,1.119228
precision,0.763158
recall,0.760216
true_negatives,652.000000
true_positives,986.000000


# Evaluation
### Evaluation: Obama dataset

In [16]:
def validate(result, estimator, test_input_fn, label_list, train, test):
    Y_pred = []
    Y_test = test['label'].tolist()
    pred_tf = estimator.predict(test_input_fn)
    for (i, prediction) in enumerate(pred_tf):
        probabilities = prediction["probabilities"]
        label = label_list[prediction["labels"]]
        Y_pred.append(label)
    from sklearn.metrics import classification_report, accuracy_score
    print(accuracy_score(np.array(Y_pred), np.array(Y_test)))
    print(classification_report(np.array(Y_pred), np.array(Y_test), digits=4))

In [17]:
validate(result_oba, estimator_oba, test_input_fn_oba, label_list_oba, train_oba, test_oba)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


0.6017777777777777
              precision    recall  f1-score   support

          -1     0.6211    0.6292    0.6252       383
           0     0.5714    0.5437    0.5572       412
           1     0.6145    0.6424    0.6281       330

   micro avg     0.6018    0.6018    0.6018      1125
   macro avg     0.6024    0.6051    0.6035      1125
weighted avg     0.6010    0.6018    0.6012      1125



### Evaluation: Romney dataset

In [18]:
validate(result_rom, estimator_rom, test_input_fn_rom, label_list_rom, train_rom, test_rom)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


0.6106194690265486
              precision    recall  f1-score   support

          -1     0.7397    0.7129    0.7261       606
           0     0.4901    0.4557    0.4723       327
           1     0.4504    0.5533    0.4966       197

   micro avg     0.6106    0.6106    0.6106      1130
   macro avg     0.5601    0.5739    0.5650      1130
weighted avg     0.6171    0.6106    0.6126      1130



### Evaluation: All dataset

In [19]:
validate(result_all, estimator_all, test_input_fn_oba, label_list_all, train_all, test_oba)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


0.8568888888888889
              precision    recall  f1-score   support

          -1     0.8892    0.8625    0.8756       400
           0     0.8597    0.8240    0.8414       409
           1     0.8174    0.8924    0.8533       316

   micro avg     0.8569    0.8569    0.8569      1125
   macro avg     0.8554    0.8596    0.8568      1125
weighted avg     0.8583    0.8569    0.8569      1125



In [20]:
validate(result_all, estimator_all, test_input_fn_rom, label_list_all, train_all, test_rom)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


0.868141592920354
              precision    recall  f1-score   support

          -1     0.9092    0.9155    0.9124       580
           0     0.8454    0.7932    0.8185       324
           1     0.7975    0.8540    0.8248       226

   micro avg     0.8681    0.8681    0.8681      1130
   macro avg     0.8507    0.8542    0.8519      1130
weighted avg     0.8686    0.8681    0.8679      1130



In [21]:
validate(result_all, estimator_all, test_input_fn_all, label_list_all, train_all, test_all)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


0.6226164079822616
              precision    recall  f1-score   support

          -1     0.6858    0.6787    0.6822       968
           0     0.5613    0.5417    0.5513       744
           1     0.5941    0.6335    0.6132       543

   micro avg     0.6226    0.6226    0.6226      2255
   macro avg     0.6137    0.6180    0.6156      2255
weighted avg     0.6226    0.6226    0.6224      2255



# Prediction

In [22]:
# load test dataset
oba_test_cleanall = np.load('./TestData/tweets_oba.npy', allow_pickle=True)
rom_test_cleanall = np.load('./TestData/tweets_rom.npy', allow_pickle=True)
oba_dummy = []
for i in range(len(oba_test_cleanall)):
    oba_dummy.append(random.randint(-1, 1))
rom_dummy = []
for i in range(len(rom_test_cleanall)):
    rom_dummy.append(random.randint(-1, 1))
df_oba_test = pd.DataFrame({'label':oba_dummy, 'text':oba_cleanall})
df_rom_test = pd.DataFrame({'label':rom_dummy, 'text':rom_cleanall})

In [23]:
# df_oba_test.head(30)

In [24]:
def tweet_predict_label(test, estimator, MAX_SEQ_LENGTH, label_list_train):
    tokenizer = create_tokenizer_from_hub_module("https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1")
    input_example = test.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x["text"],
                                                                   text_b = None,
                                                                   label = 0), axis = 1)
    label_list = test["label"].unique().tolist()
    input_features = bert.run_classifier.convert_examples_to_features(input_example, label_list, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
    pred_tf = estimator.predict(predict_input_fn)
    
    Y_pred=[]
    Y_prob=[]
    for (i, prediction) in enumerate(pred_tf):
        probabilities = prediction["probabilities"]
        labels = label_list_train[prediction["labels"]]
        Y_pred.append(labels)
        Y_prob.append(probabilities)
    return Y_pred, Y_prob

### Predict Obama test dataset and save the prediction

In [25]:
# Predict Obama test dataset with "Oba" Model
Y_oba_pred = tweet_predict_label(df_oba_test, estimator_oba, 30, label_list_oba)
with open('obama.txt', 'w') as f:
    for index, pred in enumerate(Y_oba_pred[0]):
        f.write("{};;{}\n".format(index+1, pred))
f.close()

# Predict Obama test dataset with "All" Model
Y_oba_pred_all = tweet_predict_label(df_oba_test, estimator_all, 30, label_list_all)
with open('obama_all.txt', 'w') as f:
    for index, pred in enumerate(Y_oba_pred_all[0]):
        f.write("{};;{}\n".format(index+1, pred))
f.close()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 5624


INFO:tensorflow:Writing example 0 of 5624


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Restoring parameters from output_all\model.ckpt-1690


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [26]:
len(Y_oba_pred[0])

5624

In [27]:
# print(accuracy_score(np.array(Y_oba_pred[0]), np.array(df_oba.label.values)))

### Predict Romney test dataset and save the prediction

In [28]:
# Predict Romney test dataset with "Rom" Model
Y_rom_pred = tweet_predict_label(df_rom_test, estimator_rom, 30, label_list_rom)
with open('romney.txt', 'w') as f:
    for index, pred in enumerate(Y_rom_pred[0]):
        f.write("{},,{}\n".format(index+1, pred))
f.close()

# Predict Romney test dataset with "All" Model
Y_rom_pred_all = tweet_predict_label(df_rom_test, estimator_all, 30, label_list_all)
with open('romney_all.txt', 'w') as f:
    for index, pred in enumerate(Y_rom_pred_all[0]):
        f.write("{},,{}\n".format(index+1, pred))
f.close()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 5648


INFO:tensorflow:Writing example 0 of 5648


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Restoring parameters from output_all\model.ckpt-1690


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [29]:
len(Y_rom_pred[0])

5648

In [30]:
# print(accuracy_score(np.array(Y_rom_pred[0]), np.array(df_rom.label.values)))

## utilities

In [31]:
# pip install "tensorflow>=1.15,<2.0"
# pip install --upgrade tensorflow-hub
# pip install bert-tensorflow==1.0.1
print(label_list_oba)
print(label_list_rom)
print(label_list_all)

['0', '1', '-1']
['-1', '0', '1']
['-1', '1', '0']


In [32]:
# df_oba["label"].unique().tolist()
import os
from tensorflow.python.client import device_lib 
os.environ["CUDA_VISIBLE_DEVICES"] = ""
print(device_lib.list_local_devices())
import tensorflow as tf
print(tf.__version__)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3342258747653961597
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7047741440
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8609233989608982782
physical_device_desc: "device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5"
]
1.15.0


In [33]:
# Y_pred = []
# Y_test = test['label'].tolist()
# pred_tf = estimator.predict(test_input_fn)
# label_dict = {"0":"0", "1":"1", "2":"-1"}
# target_names = ['class 0', 'class 1', 'class 2']
# for (i, prediction) in enumerate(pred_tf):
#     probabilities = prediction["probabilities"]
#     label = label_dict[str(prediction["labels"])]
#     Y_pred.append(label)
# from sklearn.metrics import classification_report, accuracy_score
# print(accuracy_score(np.array(Y_pred), np.array(Y_test)))
# print(classification_report(np.array(Y_pred), np.array(Y_test), digits=4))

In [34]:
# train, test = train_test_split(df_oba, test_size=0.2, random_state=583)

In [35]:
# tokenizer = create_tokenizer_from_hub_module("https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1")
# test_features = make_features(test, label_list, 30, tokenizer, DATA_COLUMN="text", LABEL_COLUMN="label")
# test_input_fn = run_classifier.input_fn_builder(
#         features=test_features,
#         seq_length=30,
#         is_training=False,
#         drop_remainder=False)

In [36]:
# Y_pred = []
# Y_test = test['label'].tolist()
# pred_tf = estimator.predict(test_input_fn)
# target_names = ['class 0', 'class 1', 'class 2']
# for (i, prediction) in enumerate(pred_tf):
#     probabilities = prediction["probabilities"]
#     label = label_list[prediction["labels"]]
#     Y_pred.append(label)
# from sklearn.metrics import classification_report, accuracy_score
# print(accuracy_score(np.array(Y_pred), np.array(Y_test)))
# print(classification_report(np.array(Y_pred), np.array(Y_test), digits=4))

In [37]:
# label_list
